# Mount Google Drive 
So we can access data and source code for some helper functions.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
! pip install geopandas rasterio richdem -q

     |████████████████████████████████| 972kB 2.9MB/s 
     |████████████████████████████████| 18.2MB 1.2MB/s 
     |████████████████████████████████| 4.5MB 44.6MB/s 
     |████████████████████████████████| 14.7MB 311kB/s 
     |████████████████████████████████| 10.9MB 43.7MB/s 


In [3]:
import io
import numpy as np
import geopandas as gpd
import richdem as rd
import os
import rasterio
import requests

from functools import partial
from imageio import imread
from matplotlib import pyplot as plt
from multiprocessing.pool import ThreadPool
from rasterio import transform
from scipy.ndimage.filters import convolve
from skimage import filters
from skimage.morphology import disk
from skimage.transform import resize
from skimage.util import apply_parallel


def dem_from_tnm(bbox, res, inSR=4326, **kwargs):
    """
    Retrieves a Digital Elevation Model (DEM) image from The National Map (TNM)
    web service.

    Parameters
    ----------
    bbox : list-like
      list of bounding box coordinates (minx, miny, maxx, maxy)
    res : numeric
      spatial resolution to use for returned DEM (grid cell size)
    inSR : int
      spatial reference for bounding box, such as an EPSG code (e.g., 4326)

    Returns
    -------
    dem : numpy array
      DEM image as array
    """
    width = int(abs(bbox[2] - bbox[0]) // res)
    height = int(abs(bbox[3] - bbox[1]) // res)

    BASE_URL = ''.join([
        'https://elevation.nationalmap.gov/arcgis/rest/',
        'services/3DEPElevation/ImageServer/exportImage?'
    ])

    params = dict(bbox=','.join([str(x) for x in bbox]),
                  bboxSR=inSR,
                  size=f'{width},{height}',
                  imageSR=inSR,
                  time=None,
                  format='tiff',
                  pixelType='F32',
                  noData=None,
                  noDataInterpretation='esriNoDataMatchAny',
                  interpolation='+RSP_BilinearInterpolation',
                  compression=None,
                  compressionQuality=None,
                  bandIds=None,
                  mosaicRule=None,
                  renderingRule=None,
                  f='image')
    for key, value in kwargs.items():
        params.update({key: value})

    r = requests.get(BASE_URL, params=params)
    dem = imread(io.BytesIO(r.content))

    return dem

def quad_fetch(fetcher, bbox, num_threads=4, qq=False, *args, **kwargs):
    """Breaks user-provided bounding box into quadrants and retrieves data
    using `fetcher` for each quadrant in parallel using a ThreadPool.

    Parameters
    ----------
    fetcher : callable
      data-fetching function, expected to return an array-like object
    bbox : 4-tuple or list
      coordinates of x_min, y_min, x_max, and y_max for bounding box of tile
    num_threads : int
      number of threads to use for parallel executing of data requests
    qq : bool
      whether or not to execute request for quarter quads, which executes this
      function recursively for each quadrant
    *args
      additional positional arguments that will be passed to `fetcher`
    **kwargs
      additional keyword arguments that will be passed to `fetcher`

    Returns
    -------
    quad_img : array
      image returned with quads stitched together into a single array

    """
    bboxes = split_quad(bbox)

    if qq:
        nw = quad_fetch(fetcher, bbox=bboxes[0], *args, **kwargs)
        ne = quad_fetch(fetcher, bbox=bboxes[1], *args, **kwargs)
        sw = quad_fetch(fetcher, bbox=bboxes[2], *args, **kwargs)
        se = quad_fetch(fetcher, bbox=bboxes[3], *args, **kwargs)

    else:
        get_quads = partial(fetcher, *args, **kwargs)
        with ThreadPool(num_threads) as p:
            quads = p.map(get_quads, bboxes)
            nw, ne, sw, se = quads

    quad_img = np.vstack([np.hstack([nw, ne]), np.hstack([sw, se])])

    return quad_img

def split_quad(bbox):
    """Splits a bounding box into four quadrants and returns their bounds.

    Parmeters
    ---------
    bbox : 4-tuple or list
      coordinates of x_min, y_min, x_max, and y_max for bounding box of tile

    Returns
    -------
    quads : list
      coordinates of x_min, y_min, x_max, and y_max for each quadrant, in order
      of nw, ne, sw, se
    """
    xmin, ymin, xmax, ymax = bbox
    nw_bbox = [xmin, (ymin + ymax) / 2, (xmin + xmax) / 2, ymax]
    ne_bbox = [(xmin + xmax) / 2, (ymin + ymax) / 2, xmax, ymax]
    sw_bbox = [xmin, ymin, (xmin + xmax) / 2, (ymin + ymax) / 2]
    se_bbox = [(xmin + xmax) / 2, ymin, xmax, (ymin + ymax) / 2]
    quads = [nw_bbox, ne_bbox, sw_bbox, se_bbox]

    return quads

def tpi_from_tnm(bbox,
                 irad,
                 orad,
                 dem_resolution,
                 smooth_highres_dem=True,
                 tpi_resolution=30,
                 parallel=True,
                 norm=True,
                 fixed_mean=None,
                 fixed_std=None,
                 **kwargs):
    """
    Produces a raster of Topographic Position Index (TPI) by fetching a Digital
    Elevation Model (DEM) from The National Map (TNM) web service.

    TPI is the difference between the elevation at a location from the average
    elevation of its surroundings, calculated using an annulus (ring). This
    function permits the calculation of average surrounding elevation using
    a coarser grain, and return the TPI user a higher-resolution DEM.

    Parameters
    ----------
    bbox : list-like
      list of bounding box coordinates (minx, miny, maxx, maxy)
    irad : numeric
      inner radius of annulus used to calculate TPI
    orad : numeric
      outer radius of annulus used to calculate TPI
    dem_resolution : numeric
      spatial resolution of Digital Elevation Model (DEM)
    tpi_resolution : numeric
      spatial resolution of DEM used to calculate TPI
    norm : bool
      whether to return a normalized version of TPI, with mean = 0 and SD = 1
    fixed_mean : numeric
      mean value to use to normalize data, useful to to set as a constant when
      processing adjacent tiles to avoid stitching/edge effects
    fixed_std : numeric
      standard deviation value to use to normalize data, useful to to set as a
      constant when processing adjacent tiles to avoid stitching/edge effects

    Returns
    -------
    tpi : array
      TPI image as array
    """
    tpi_bbox = np.array(bbox)
    tpi_bbox[0:2] = tpi_bbox[0:2] - orad
    tpi_bbox[2:4] = tpi_bbox[2:4] + orad
    k_orad = orad // tpi_resolution
    k_irad = irad // tpi_resolution

    kernel = disk(k_orad) - np.pad(disk(k_irad), pad_width=(k_orad - k_irad))
    weights = kernel / kernel.sum()

    if dem_resolution != tpi_resolution:
        dem = dem_from_tnm(bbox, dem_resolution, **kwargs)
        if dem_resolution < 3 and smooth_highres_dem:
            dem = filters.gaussian(dem, 3)
        dem = np.pad(dem, orad // dem_resolution)
        tpi_dem = dem_from_tnm(tpi_bbox, tpi_resolution, **kwargs)

    else:
        tpi_dem = dem_from_tnm(tpi_bbox, tpi_resolution, **kwargs)
        dem = tpi_dem

    if parallel:

        def conv(tpi_dem):
            return convolve(tpi_dem, weights)

        convolved = apply_parallel(conv, tpi_dem, compute=True, depth=k_orad)
        if tpi_resolution != dem_resolution:
            tpi = dem - resize(convolved, dem.shape)
        else:
            tpi = dem - convolved

    else:
        if tpi_resolution != dem_resolution:
            tpi = dem - resize(convolve(tpi_dem, weights), dem.shape)
        else:
            tpi = dem - convolve(tpi_dem, weights)

    # trim the padding around the dem used to calculate TPI
    tpi = tpi[orad // dem_resolution:-orad // dem_resolution,
              orad // dem_resolution:-orad // dem_resolution]

    if norm:
        if fixed_mean is not None and fixed_std is not None:
            tpi_mean = fixed_mean
            tpi_std = fixed_std
        else:
            tpi_mean = (tpi_dem - convolved).mean()
            tpi_std = (tpi_dem - convolved).std()
        tpi = (tpi - tpi_mean) / tpi_std

    return tpi

# Download Data for Training Tiles
We have prepared shapefiles containing the USGS quarter quadrangles that have good coverage of forest stand delineations that we want to grab other data for. We'll fetch elevation data (a Digital Elevation Model) from The National Map for each tile, create some additional derivative products, and write our outputs as GeoTiffs to Google Drive. 

In [4]:
import glob

In [5]:
WORK_DIR = '/content/drive/Shared drives/stand_mapping/data/processed/training_tiles/'
OR_QUADS = ['oregon_utm10n_training_quads_epsg6339.shp', 'oregon_utm11n_training_quads_epsg6340.shp']
WA_QUADS = ['washington_utm10n_training_quads_epsg6339.shp', 'washington_utm11n_training_quads_epsg6340.shp']

In [6]:
len(glob.glob(os.path.join(WORK_DIR, 'or_training_tiles', '*_tpi300.tif')))

762

In [7]:
def fetch_dems(path_to_tiles, out_dir, overwrite=False):
    gdf = gpd.read_file(path_to_tiles)
    epsg = gdf.crs.to_epsg()
    print('Fetching DEMs for {:,d} tiles'.format(len(gdf)))
    
    PROFILE = {
        'driver': 'GTiff',
        'interleave': 'band',
        'tiled': True,
        'blockxsize': 256,
        'blockysize': 256,
        'compress': 'lzw',
        'nodata': -9999,
        'dtype': rasterio.float32,
        'count': 1,
        }

    ## loop through all the geometries in the geodataframe and fetch the DEM

    for idx, row in gdf.iterrows():
        xmin, ymin, xmax, ymax = row['geometry'].bounds
        xmin, ymin = np.floor((xmin, ymin))
        xmax, ymax = np.ceil((xmax, ymax))

        width, height = xmax-xmin, ymax-ymin
        trf = transform.from_bounds(xmin, ymin, xmax, ymax, width, height)

        ## don't bother fetching data if we already have processed this tile
        outname = f'{row.CELL_ID}_dem.tif'
        outfile = os.path.join(out_dir, outname)        
        if os.path.exists(outfile) and not overwrite:
            if idx % 100 == 0:
                print()
            if idx % 10 == 0:
                print(idx, end='')
            else:
                print('.', end='')
            continue
            
        dem = quad_fetch(dem_from_tnm, 
                         bbox=[xmin, ymin, xmax, ymax], 
                         qq=True, res=1, inSR=epsg, noData=-9999)
        
        ## apply a smoothing filter to mitigate stitching/edge artifacts
        dem = filters.gaussian(dem, 3)

        
        ## write the data to disk
        PROFILE.update(width=width, height=height)

        with rasterio.open(outfile, 'w', 
                           **PROFILE, crs=epsg, transform=trf) as dst:
            dst.write(dem.astype(rasterio.float32), 1)
            dst.set_band_unit(1, 'meters')
            dst.set_band_description(1, 'DEM retrieved from The National Map')
        
        ## report progress
        if idx % 100 == 0:
            print()
        if idx % 10 == 0:
            print(idx, end='')
        else:
            print('.', end='')

In [8]:
def fetch_tpis(path_to_tiles, out_dir, overwrite=False):
    gdf = gpd.read_file(path_to_tiles)
    epsg = gdf.crs.to_epsg()
    print('Fetching TPIs for {:,d} tiles'.format(len(gdf)))
    
    PROFILE = {
        'driver': 'GTiff',
        'interleave': 'band',
        'tiled': True,
        'blockxsize': 256,
        'blockysize': 256,
        'compress': 'lzw',
        'nodata': -9999,
        'dtype': rasterio.float32,
        'count': 1,
        }

    ## loop through all the geometries in the geodataframe and fetch the DEM

    for idx, row in gdf.iterrows():
        xmin, ymin, xmax, ymax = row['geometry'].bounds
        xmin, ymin = np.floor((xmin, ymin))
        xmax, ymax = np.ceil((xmax, ymax))

        width, height = xmax-xmin, ymax-ymin
        trf = transform.from_bounds(xmin, ymin, xmax, ymax, width, height)

        ## don't bother fetching data if we already have processed this tile
        outname300 = f'{row.CELL_ID}_tpi300.tif'
        outname2000 = f'{row.CELL_ID}_tpi2000.tif'
        outfile300 = os.path.join(out_dir, outname300)        
        outfile2000 = os.path.join(out_dir, outname2000)        
        if os.path.exists(outfile300) and os.path.exists(outfile2000) and not overwrite:
            if idx % 100 == 0:
                print()
            if idx % 10 == 0:
                print(idx, end='')
            else:
                print('.', end='')
            continue
            
        tpi300 = quad_fetch(tpi_from_tnm, bbox=[xmin, ymin, xmax, ymax],
                            qq=True, irad=150, orad=300, dem_resolution=1, 
                            norm=False, inSR=epsg, noData=-9999)

        tpi2000 = quad_fetch(tpi_from_tnm, bbox=[xmin, ymin, xmax, ymax],
                             qq=True, irad=1850, orad=2000, dem_resolution=1, 
                             norm=False, inSR=epsg, noData=-9999)
        
        ## write the data to disk
        PROFILE.update(width=width, height=height, crs=epsg, transform=trf)

        DESC300 = ''.join(['Topographic Position Index for 150-300 meter',
                           'annulus calculated from DEM retrieved from The',
                           'National Map'])
        DESC2000 = ''.join(['Topographic Position Index for 1850-2000 meter',
                            'annulus calculated from DEM retrieved from The',
                            'National Map'])

        with rasterio.open(outfile300, 'w', **PROFILE) as dst:
                           dst.write(tpi300.astype(rasterio.float32), 1)
                           dst.set_band_description(1, DESC300)
        
        with rasterio.open(outfile2000, 'w', **PROFILE) as dst:
                    dst.write(tpi2000.astype(rasterio.float32), 1)
                    dst.set_band_description(1, DESC2000)
        
        ## report progress
        if idx % 100 == 0:
            print()
        if idx % 10 == 0:
            print(idx, end='')
        else:
            print('.', end='')

In [13]:
fetch_dems(path_to_tiles=os.path.join(WORK_DIR, WA_QUADS[1]),
           out_dir=os.path.join(WORK_DIR, 'wa_training_tiles'),
           overwrite=False)

Fetching DEMs for 82 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.

In [ ]:
fetch_tpis(path_to_tiles=os.path.join(WORK_DIR, OR_QUADS[1]),
           out_dir=os.path.join(WORK_DIR, 'or_training_tiles'),
           overwrite=False)

Fetching TPIs for 524 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250..

In [ ]:
fetch_tpis(path_to_tiles=os.path.join(WORK_DIR, OR_QUADS[1]),
           out_dir=os.path.join(WORK_DIR, 'or_training_tiles'),
           overwrite=True)

In [ ]:
fetch_data(path_to_tiles = os.path.join(WORK_DIR, OR_QUADS[1]), 
           out_dir = os.path.join(WORK_DIR, 'or_training_tiles'), overwrite=True)

In [ ]:
fetch_data(path_to_tiles = os.path.join(WORK_DIR, WA_QUADS[0]), 
           out_dir = os.path.join(WORK_DIR, 'wa_training_tiles'), overwrite=True)

In [ ]:
fetch_data(path_to_tiles = os.path.join(WORK_DIR, WA_QUADS[1]), 
           out_dir = os.path.join(WORK_DIR, 'wa_training_tiles'), overwrite=True)